# Washington State DOT Traffic API

source: https://wsdot.wa.gov/traffic/api/


In [1]:
import pandas as pd
import requests

from pathlib import Path

## Import env variables


In [2]:
from dotenv import load_dotenv
import os

# load the .env file
load_dotenv()

# access the env variables
api_key = os.getenv("API_KEY")

# print to verify
# print(f"API Key: {api_key}")

## URLs to Access API


In [3]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

### Function to Get API data


In [4]:
def get_api_data(url, access_key):
    """_summary_

    Args:
        url (_type_): _description_
        access_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code

In [7]:
def get_updates(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(traffic_alerts_url, access_key)
    df_wa, response_wa = get_api_data(weather_information_url, access_key)

    # add pull_date_time
    df_tt["timestamp"] = timestamp
    df_ta["timestamp"] = timestamp
    df_wa["timestamp"] = timestamp

    # assign column to use as foreign key
    df_tt["tt_id"] = 1
    df_ta["ta_id"] = 2
    df_wa["wa_id"] = 3

    df_response = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
                "tt_id": 1,
                "ta_id": 2,
                "wa_id": 3,
            }
        ]
    )

    return df_response, df_tt, df_ta, df_wa

## Travel Times API


In [25]:
df_travel_times, reponse = get_api_data(TRAVEL_TIMES_URL, api_key)
df_travel_times.head()

Data fetched successfully.


,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID
0,25,25,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,Everett-Seattle HOV,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1738219800000-0800)/,2
1,25,25,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett HOV,{'Description': 'I-5 @ University St in Seattl...,/Date(1738219800000-0800)/,3
2,25,25,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett,{'Description': 'I-5 @ University St in Seattl...,/Date(1738219800000-0800)/,4
3,9,9,Downtown Bellevue to Issaquah,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738219800000-0800)/,5
4,9,9,Downtown Bellevue to Issaquah using HOV lanes,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738219800000-0800)/,6


## Traffic Alerts API


In [26]:
df_traffic_alerts, response = get_api_data(TRAFFIC_ALERTS_URL, api_key)
df_traffic_alerts.head()

Data fetched successfully.


,AlertID,County,EndRoadwayLocation,EndTime,EventCategory,EventStatus,ExtendedDescription,HeadlineDescription,LastUpdatedTime,Priority,Region,StartRoadwayLocation,StartTime
0,624112,Grant,"{'Description': None, 'Direction': 'B', 'Latit...",None,Rest Area,Open,,The blue Lake rest area is closed for the winter.,/Date(1731629066113-0800)/,Low,North Central,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1731628800000-0800)/
1,632850,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738328400000-0800)/,Maintenance,Open,,"Nightly from 9 p.m. until 5 a.m. Sunday, Jan. ...",/Date(1737765776500-0800)/,High,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737765300000-0800)/
2,632245,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738371600000-0800)/,Maintenance,Open,,"Weekdays, from 7 a.m. to 5 p.m., Wednesday, Ja...",/Date(1737148405100-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737148080000-0800)/
3,632835,None,"{'Description': None, 'Direction': 'B', 'Latit...",None,Construction,Open,,Eastbound and westbound SR 18 will shift to th...,/Date(1737761204897-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738155600000-0800)/
4,632782,None,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1738152000000-0800)/,Construction,Open,,"Tuesday, Jan. 27 to the morning of Wednesday, ...",/Date(1737738428410-0800)/,Medium,Northwest,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1737738900000-0800)/


## Weather Information API


In [27]:
df_weather_alerts, response = get_api_data(WEATHER_INFORMATION_URL, api_key)
df_weather_alerts.head()

Data fetched successfully.


,BarometricPressure,Latitude,Longitude,PrecipitationInInches,ReadingTime,RelativeHumidity,SkyCoverage,StationID,StationName,TemperatureInFahrenheit,Visibility,WindDirection,WindDirectionCardinal,WindGustSpeedInMPH,WindSpeedInMPH
0,960.3,47.474800,-122.270400,None,/Date(1738219505000-0800)/,99.0,N/A,1909,S 144th St on SB I-5 at mp 155.32,34.88,1.0,175.0,S,0.0,0.0
1,1040.9,47.760633,-122.184048,None,/Date(1738219515000-0800)/,97.0,N/A,1910,NE 195th on SB I-405 at mp 24.58,29.30,1.0,5.0,N,0.0,0.0
2,984.7,47.509000,-121.885000,None,/Date(1738219506000-0800)/,82.0,N/A,1928,EB I-90 / SR-18 (Echo Lake) at mp 26.30,31.28,1.0,NaN,N/A,1.0,0.0
3,1006.8,47.726000,-122.324000,None,/Date(1738219502000-0800)/,97.0,N/A,1966,NE 130th Street on I-5 at mp 173.75,33.26,1.0,85.0,E,0.0,0.0
4,NaN,46.436000,-117.350000,None,/Date(1738219054000-0800)/,64.0,N/A,1968,Alpowa Summit on US 12 at mp 413.36,28.22,12.0,166.0,SSE,6.0,5.0


# Call to get all info from API


In [8]:
df_response, df_tt, df_ta, df_wa = get_updates(
    TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.


# Export to CSV


In [10]:
# Set path to datasets folder
datasets = Path("../datasets")

In [11]:
# Save dataframes to csv
df_response.to_csv(datasets / "api_fetch.csv", index=False)
df_tt.to_csv(datasets / "travel_times.csv", index=False)
df_ta.to_csv(datasets / "traffic_alerts.csv", index=False)
df_wa.to_csv(datasets / "weather_alerts.csv", index=False)

In [50]:
df_response

,current_date_time,travel_times_response,traffic_alerts_response,weather_alerts_response
0,2025-01-29 09:52:23.041360,<Response [200]>,<Response [200]>,<Response [200]>


In [12]:
df_tt

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID,timestamp,tt_id
0,46,34,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,Everett-Seattle HOV,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1738280100000-0800)/,2,2025-01-30 15:37:13.277193,1
1,38,33,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett HOV,{'Description': 'I-5 @ University St in Seattl...,/Date(1738280100000-0800)/,3,2025-01-30 15:37:13.277193,1
2,57,43,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett,{'Description': 'I-5 @ University St in Seattl...,/Date(1738280100000-0800)/,4,2025-01-30 15:37:13.277193,1
3,16,15,Downtown Bellevue to Issaquah,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738280100000-0800)/,5,2025-01-30 15:37:13.277193,1
4,13,10,Downtown Bellevue to Issaquah using HOV lanes,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738280100000-0800)/,6,2025-01-30 15:37:13.277193,1
...,...,...,...,...,...,...,...,...,...,...,...
163,8,5,SR500 WB and 112th Ave to SR500 and I-5 JCT,4.80,{'Description': 'SR500 and I-5 JCT in Vancouve...,021 SR500 WB and 112th Ave to SR500 and I-5 JCT,{'Description': 'SR500 WB and 112th Ave in Van...,/Date(1738280100000-0800)/,528,2025-01-30 15:37:13.277193,1
164,19,23,SR500 WB and 112th Ave to I-5 to I-5 at I-84 JCT,14.16,"{'Description': 'I-5 at I-84 JCT in Portland',...",021 SR500 WB and 112th Ave to I-5 to I-5 at I-...,{'Description': 'SR500 WB and 112th Ave in Van...,/Date(1738280100000-0800)/,529,2025-01-30 15:37:13.277193,1
165,20,21,SR500 WB and 112th Ave to I-205 to I-84 to I-5...,17.42,"{'Description': 'I-84 to I-5 JCT in Portland',...",022 SR500 WB and 112th Ave to I-205 to I-84 to...,{'Description': 'SR500 WB and 112th Ave in Van...,/Date(1738280100000-0800)/,530,2025-01-30 15:37:13.277193,1
166,16,24,SB I-5 54th Ave To JBLM Main Gate,17.10,"{'Description': 'I-5 @ JBLM Main Gate', 'Direc...","SB I-5, Fife To JBLM Main Gate","{'Description': 'I-5 @ 54th Ave', 'Direction':...",/Date(1738280100000-0800)/,531,2025-01-30 15:37:13.277193,1


In [13]:
df_ta

,AlertID,County,EndRoadwayLocation,EndTime,EventCategory,EventStatus,ExtendedDescription,HeadlineDescription,LastUpdatedTime,Priority,Region,StartRoadwayLocation,StartTime,timestamp,ta_id
0,624112,Grant,"{'Description': None, 'Direction': 'B', 'Latit...",None,Rest Area,Open,,The blue Lake rest area is closed for the winter.,/Date(1731629066113-0800)/,Low,North Central,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1731628800000-0800)/,2025-01-30 15:37:13.277193,2
1,632850,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738328400000-0800)/,Maintenance,Open,,"Nightly from 9 p.m. until 5 a.m. Sunday, Jan. ...",/Date(1737765776500-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737765300000-0800)/,2025-01-30 15:37:13.277193,2
2,634326,Clark,"{'Description': None, 'Direction': 'W', 'Latit...",None,Maintenance,Open,,Maintenance on SR 14 westbound from milepost 6...,/Date(1738263238390-0800)/,Medium,Southwest,"{'Description': None, 'Direction': 'W', 'Latit...",/Date(1738263120000-0800)/,2025-01-30 15:37:13.277193,2
3,632245,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738371600000-0800)/,Maintenance,Open,,"Weekdays, from 7 a.m. to 5 p.m., Wednesday, Ja...",/Date(1737148405100-0800)/,Medium,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737148080000-0800)/,2025-01-30 15:37:13.277193,2
4,632835,None,"{'Description': None, 'Direction': 'B', 'Latit...",None,Construction,Open,,Eastbound and westbound SR 18 will shift to th...,/Date(1737761204897-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738155600000-0800)/,2025-01-30 15:37:13.277193,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,608908,None,"{'Description': None, 'Direction': 'W', 'Latit...",None,Construction,Open,,Expect congestion in the right lanes of westbo...,/Date(1723048475597-0700)/,Low,Northwest,"{'Description': None, 'Direction': 'W', 'Latit...",/Date(1722980580000-0700)/,2025-01-30 15:37:13.277193,2
145,632770,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738242000000-0800)/,Maintenance,Open,,"Nightly, from 8 p.m. to 5 a.m. Tuesday, Jan. 2...",/Date(1737735511457-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737733260000-0800)/,2025-01-30 15:37:13.277193,2
146,626561,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1735444800000-0800)/,Heavy traffic is causing backups and possible ...,Open,,On US 2 between milepost 99 and 101 (city of L...,/Date(1732905135207-0800)/,Low,North Central,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1732904160000-0800)/,2025-01-30 15:37:13.277193,2
147,634137,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738155600000-0800)/,Construction,Open,,"From 8 p.m. Tuesday, Jan. 28 to 6 a.m. Wednesd...",/Date(1738083970563-0800)/,High,Olympic,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738083660000-0800)/,2025-01-30 15:37:13.277193,2


In [14]:
df_wa

,BarometricPressure,Latitude,Longitude,PrecipitationInInches,ReadingTime,RelativeHumidity,SkyCoverage,StationID,StationName,TemperatureInFahrenheit,Visibility,WindDirection,WindDirectionCardinal,WindGustSpeedInMPH,WindSpeedInMPH,timestamp,wa_id
0,958.8,47.474800,-122.270400,None,/Date(1738278916000-0800)/,99.0,N/A,1909,S 144th St on SB I-5 at mp 155.32,44.60,1.0,150.0,SSE,5.0,1.0,2025-01-30 15:37:13.277193,3
1,999.1,47.760633,-122.184048,None,/Date(1738278925000-0800)/,71.0,N/A,1910,NE 195th on SB I-405 at mp 24.58,43.16,1.0,5.0,N,0.0,0.0,2025-01-30 15:37:13.277193,3
2,982.8,47.509000,-121.885000,None,/Date(1738278917000-0800)/,70.0,N/A,1928,EB I-90 / SR-18 (Echo Lake) at mp 26.30,40.64,1.0,232.0,SW,4.0,0.0,2025-01-30 15:37:13.277193,3
3,1004.0,47.726000,-122.324000,None,/Date(1738278914000-0800)/,75.0,N/A,1966,NE 130th Street on I-5 at mp 173.75,41.90,1.0,90.0,E,4.0,3.0,2025-01-30 15:37:13.277193,3
4,NaN,46.436000,-117.350000,None,/Date(1738279353000-0800)/,47.0,N/A,1968,Alpowa Summit on US 12 at mp 413.36,37.94,12.0,162.0,SSE,8.0,6.0,2025-01-30 15:37:13.277193,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,985.2,46.571410,-118.988510,None,/Date(1738278577000-0800)/,57.0,N/A,5673,Mesa/SR17 Interchange,38.66,1.0,NaN,N/A,2.0,1.0,2025-01-30 15:37:13.277193,3
99,NaN,47.231001,-122.457407,None,/Date(1738279646000-0800)/,67.0,N/A,5678,I5 SB at SR 16 I/C Flyover,44.60,1.0,182.0,S,6.0,3.0,2025-01-30 15:37:13.277193,3
100,NaN,47.231301,-122.461009,None,/Date(1738279666000-0800)/,NaN,N/A,5680,SR 16 at I5 I/C HOV,NaN,NaN,NaN,N/A,NaN,NaN,2025-01-30 15:37:13.277193,3
101,999.0,46.054560,-118.667690,None,/Date(1738279377000-0800)/,58.0,N/A,5883,Touchet Road Interchange,40.10,1.0,NaN,N/A,2.0,1.0,2025-01-30 15:37:13.277193,3
